In [ ]:
# Mounting Google Colab drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report, recall_score, precision_score
from joblib import Parallel, delayed
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
# Installing required packages
!pip install torch-geometric

import torch_geometric.transforms as T
from torch_geometric.nn import GATConv, Linear, to_hetero
from torch_geometric.data import HeteroData

# Change directory to location
loc = "/content/drive/MyDrive/KE_GNN/"
os.chdir(loc)
os.getcwd()

#number of epochs
epoch_n = 411
#sample number
run_number = 1

def move_to_device(obj, device):
  '''
  moves a dictionary to device (if needed)
  '''
  if isinstance(obj, torch.Tensor):
      return obj.to(device)
  elif isinstance(obj, dict):
      return {k: move_to_device(v, device) for k, v in obj.items()}
  elif isinstance(obj, list):
      return [move_to_device(i, device) for i in obj]
  elif isinstance(obj, tuple):
      return tuple(move_to_device(i, device) for i in obj)
  elif isinstance(obj, set):
      return {move_to_device(i, device) for i in obj}
  else:
      return obj

# load train graph and clause dictionary
data_train = torch.load('{}Graph storage/train_graph.pt'.format(loc))
with open('{}Clause Storage/train_KE_location_large.pkl'.format(loc), 'rb') as handle:
    train_KE_location = pickle.load(handle)

# load train graph and clause dictionary
data_valid = torch.load('{}Graph storage/valid_graph.pt'.format(loc))
with open('{}Clause Storage/valid_KE_location_large.pkl'.format(loc), 'rb') as handle:
    valid_KE_location = pickle.load(handle)

# load train graph and clause dictionary
data_test = torch.load('{}Graph storage/test_graph.pt'.format(loc))
with open('{}Clause Storage/test_KE_location_large.pkl'.format(loc), 'rb') as handle:
    test_KE_location = pickle.load(handle)

# load knowledge enhancement
with open('{}Clause Storage/Knowledge_enhancements_large.pkl'.format(loc), 'rb') as handle:
    KE_conditions = pickle.load(handle)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# only keeping 5 of the top clauses
def subset_dict(original_dict, keys_for_subset):
    new_dict = {key: original_dict[key] for key in keys_for_subset if key in original_dict}
    print(new_dict.keys())
    return new_dict

keys_keep = ['RULE1', 'RULE2', 'RULE3', 'RULE4', 'RULE5']
train_KE_location = subset_dict(train_KE_location, keys_keep)
valid_KE_location = subset_dict(valid_KE_location, keys_keep)
test_KE_location = subset_dict(test_KE_location, keys_keep)
KE_conditions = subset_dict(KE_conditions, keys_keep)

# GAT model definition
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        head = 2
        HC = hidden_channels * head
        # First graph attention layer
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False, heads=head)
        self.lin1 = Linear(-1, HC)

        # Second graph attention layer
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x

# Define the outer GNN model
class OuterGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, KE_dictionary):
        super().__init__()
        self.gat = GAT(hidden_channels, out_channels)
        self.gat = to_hetero(self.gat, metadata, aggr='sum')
        self.lin = Linear(-1, out_channels)

        self.params_conditions = nn.ParameterDict()
        for k,v in KE_dictionary.items():
          self.params_conditions[k] = nn.Parameter(torch.tensor([1], dtype=torch.float32))

    def forward(self, data, conditions):
        # loading data into GAT model
        x = self.gat(data.x_dict, data.edge_index_dict)
        # sigmoid out function
        x = torch.sigmoid(x['transaction'])

        final_out = x
        rule_outputs = []
        KE_output_dic = {}
        for k, v in conditions.items():
          #ensuring that only conditions with at least one modification is run through
          if torch.sum(v) > 0:
            # dictionary value for each condition and the modification
            KE_output_dic[k] = ((self.params_conditions[k]-1) * v) * final_out
            # appending the clause weights
            rule_outputs.append((k,self.params_conditions[k].item()))
            #final modification to the output function, ensuring it remains between 0-1
            final_out = torch.clamp(KE_output_dic[k] + final_out, min = 0, max= 1)
        return final_out, rule_outputs

# moving data to device if using GPU training
data_valid = data_valid.to(device)
data_test = data_test.to(device)
data_train = data_train.to(device)
metadata = data_train.metadata()

# moving clause location to device if using GPU training
KE_conditions = move_to_device(KE_conditions, device)
train_KE_location = move_to_device(train_KE_location, device)
valid_KE_location = move_to_device(valid_KE_location, device)
test_KE_location = move_to_device(test_KE_location, device)


# Instantiate the outer model
hidden_channels = 120
out_channels = 1
model = OuterGNN(hidden_channels, out_channels, metadata, KE_conditions)

criterion = torch.nn.BCELoss()

#criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = model.to(device)

with torch.no_grad():  # Initialize lazy modules.
    out, _ = model(data_train, train_KE_location)


def inductive_train():
    '''
      Performs a single training step for the model.
    Returns:
        loss: The computed loss for the current training step.
        rule_outputs: The output from clause weights
    '''
    model.train()
    optimizer.zero_grad()  # Clear gradients
    out, rule_outputs = model(data_train, train_KE_location)  # Perform a single forward pass
    loss = criterion(out, data_train['transaction'].y)  # Compute the loss solely based on the training nodes
    loss.backward()
    optimizer.step()
    return loss, rule_outputs



def f1_finder(pred, true, max_val):
    '''
    Finds the best threshold for maximizing the F1-score of a binary classifier.

    Args:
        pred: Predicted values for the positive class.
        true: True binary labels.
        max_val: The maximum threshold value to consider.

    Returns:
        The threshold that maximizes the F1-score.
    '''
    thresholds = np.linspace(0, max_val, num=200, endpoint=True)

    def compute_f1(threshold):
        return f1_score(true, (pred > threshold).astype(int), zero_division=0.0)

    f1_scores = Parallel(n_jobs=-1)(delayed(compute_f1)(x) for x in thresholds)

    best_index = np.argmax(f1_scores)
    best_x = thresholds[best_index]
    return best_x


def test():
    '''
    Test the model on the validation set.

    Returns:
        The predicted and true labels for the validation set.
    '''
    model.eval()
    out, _ = model(data_valid, valid_KE_location)
    pred = out.detach().cpu().numpy()
    true_labels = data_valid['transaction'].y.cpu()
    return pred, true_labels.numpy()



pr_best = 0
prd = 0
best_model_state = None
measures = []
weights = []

for epoch in range(1, epoch_n):
  print(epoch)
  loss, rule_outputs = inductive_train()
  weights.append(rule_outputs.copy())
  if epoch % 10 == 0:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
  if epoch % 10 == 0:
      pred, truess = test()

      threshold = f1_finder(pred, truess, 1.0)
      predss_thres = (pred > threshold).astype(int)

      pr = f1_score(truess, predss_thres)
      print('Best current threshold:', threshold, 'Best F1 score:', pr, ' number of fraud: ', np.sum(truess))
      recall = recall_score(truess, predss_thres, zero_division = 0.0)
      prc = precision_score(truess, predss_thres, zero_division = 0.0)
      measures.append([epoch,loss,pr,threshold, recall,prc])

      if pr > pr_best:
        print('new best model')
        pr_best = pr  # Update the best F1 score
        best_thresh = threshold

        torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss, }, '{}/model_storage/KeGAT5_{}.pt'.format(loc,run_number))

print('final best f1: {}, best threshold: {}'.format(pr_best, best_thresh))


print(weights)
extracted_data = []
# Extract tensor values from the list
for rule_set in weights:
    row = {rule: param for rule, param in rule_set}
    extracted_data.append(row)
df3 = pd.DataFrame(extracted_data)

hidden_channels = 120
out_channels = 1
model = OuterGNN(hidden_channels, out_channels, metadata, KE_conditions)

criterion = torch.nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = model.to(device)
checkpoint = torch.load('{}/model_storage/KeGAT5_{}.pt'.format(loc,run_number))


# Load the model and optimizer state dictionaries
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# Set the model to evaluation mode

model.eval()

test_measure = []
out, _ = model(data_test, test_KE_location)
pred = out.detach().cpu().numpy()
true_labels = data_test['transaction'].y.cpu()
# predictions using the best threshold found in the best model
predss_thres = (pred > best_thresh).astype(int)
best_validation_threshold = best_thresh.copy()
# f1 score from the test
f1_val_thresh_test_set = f1_score(true_labels, predss_thres)
print('validation optimised test results:')
print(classification_report(true_labels, predss_thres))
recall_val_thres_test = recall_score(true_labels, predss_thres)
precision_val_thres_test = precision_score(true_labels, predss_thres)

# test optimised results
threshold = f1_finder(pred, true_labels, 1.0)
test_optimised_prediction = (pred > threshold).astype(int)

f1_test_optimised = f1_score(true_labels, test_optimised_prediction, zero_division = 0.0)
recall_test_optimised = recall_score(true_labels, test_optimised_prediction, zero_division = 0.0)
precision_test_optimised = precision_score(true_labels, test_optimised_prediction, zero_division = 0.0)

test_measure.append([f1_val_thresh_test_set, best_validation_threshold, recall_val_thres_test, precision_val_thres_test,
                   f1_test_optimised, threshold, recall_test_optimised, precision_test_optimised])

# final model results
df2 = pd.DataFrame(test_measure, columns = ['test_train_thresh_f1', 'test_train_thresh', 'test_train_recall', 'test_train_precision',
                                          'test_f1','test_thresh', 'test_recall', 'test_precision'])
#training results
df1 = pd.DataFrame(measures, columns=['epoch', 'training loss', 'optimised_f1','threshold', 'recall', 'precision'])


df1.to_csv('{}/Main Experiment/output/KeGAT_5_clause/KE_GAT_training_results_nest{}.csv'.format(loc,run_number))
df2.to_csv('{}/Main Experiment/output/KeGAT_5_clause/KE_GAT_test_results_nest{}.csv'.format(loc,run_number))
df3.to_csv('{}/Main Experiment/output/KeGAT_5_clause/KE_GAT_train_weights_nest{}.csv'.format(loc,run_number))

